# Zillow Home Value Prediction 

#### Author - Danish Anis
#### Contact - danishanis10@hotmail.com

## Part 1 - DEFINE

### ---- 1 Defining the problem ----

A home is often the largest and most expensive purchase a person makes in his or her lifetime. Ensuring homeowners have a trusted way to monitor this asset is incredibly important. The Zestimate was created to give consumers as much information as possible about homes and the housing market, marking the first time consumers had access to this type of home value information at no cost.

This analysis is aimed at pushing the accuracy of the Zestimate even further by developing an algorithm that makes predictions about the future sale prices of homes. **The objective is to develop a model that can make price predictions about properties in different time periods.** Therefore, we are to take each '*ParcelID*' and make predictions for the stated 6 time periods.

The data we'll be using is the [Zillow Home Price](https://www.kaggle.com/c/zillow-prize-1/data) dataset that was used in a Kaggle competition.

In [66]:
#Libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline



In [64]:
#Functions





## Part 2 - DISCOVER 

### ---- 2 Getting the raw data ----

In [1]:
#Finding the working directory
!dir -lh raw_data

 Volume in drive C has no label.
 Volume Serial Number is FC01-E7A1

 Directory of C:\Users\TOSHIBA\Zillow_Prediction_Project


 Directory of C:\Users\TOSHIBA\Zillow_Prediction_Project\raw_data

12/30/2019  03:35 AM    <DIR>          .
12/30/2019  03:35 AM    <DIR>          ..
12/30/2019  03:38 AM       171,432,613 properties_2016.csv.zip
12/30/2019  03:38 AM       172,874,882 properties_2017.csv.zip
12/30/2019  03:35 AM           658,833 train_2016_v2.csv.zip
12/30/2019  03:35 AM           964,166 train_2017.csv.zip
12/30/2019  03:35 AM            19,229 zillow_data_dictionary.xlsx
               5 File(s)    345,949,723 bytes
               2 Dir(s)  347,621,466,112 bytes free


File Not Found


In [2]:
%%bash
#turns the whole cell into a bash script.

# make a directory called unzipped_data, deleting one if it exists
rm -rf unzipped_data
mkdir unzipped_data

# unzip the raw data into a directory called unzipped_data
unzip "raw_data/*.zip" -d unzipped_data

# rename train_2016_v2.csv to train_2016.csv. there is no V1, so it's all the same to us
mv unzipped_data/train_2016_v2.csv unzipped_data/train_2016.csv

mesg: ttyname failed: Inappropriate ioctl for device
-bash: line 8: unzip: command not found
mv: cannot stat 'unzipped_data/train_2016_v2.csv': No such file or directory


In [3]:
%%bash 


conda install pandas xlrd matplotlib scikit-learn

mesg: ttyname failed: Inappropriate ioctl for device
-bash: line 3: conda: command not found


### ---- 3 Preparing Data ---- 

Creating a helpers funciton helps save time as it can be used in another notebook. We'll put these in a file called helpers.py in the scripts directory in the root of our project. Also, any neccessary changes in the function need to be performed only once.

The helpers.py file can be found in the modules directory.

In [5]:
#adding script directory to python (set of places where the python interpreter looks for files)
import sys
sys.path.insert(0, './scripts')

#reading the functions
from helpers import read_in_dataset, merge_dataset

Viewing the source code of the function in the notebook

In [6]:
#inline
import inspect
print(inspect.getsource(read_in_dataset))

def read_in_dataset(dset, verbose=False):
    
    """Read in one of the Zillow datasets (train or properties)

    Keyword arguments:
    dset -- a string in {properties_2016, properties_2017, train_2016, train_2017}
    verbose -- whether or not to print info about the dataset
    
    Returns:
    a pandas dataframe
    """
    
    df = pd.read_csv('unzipped_data/{0}.csv'.format(dset))
    
    if verbose:
        print('\n{0:*^80}'.format(' Reading in the {0} dataset '.format(dset)))
        print("\nit has {0} rows and {1} columns".format(*df.shape))
        print('\n{0:*^80}\n'.format(' It has the following columns '))
        print(df.columns)
        print('\n{0:*^80}\n'.format(' The first 5 rows look like this '))
        print(df.head())
        
    return df



In [7]:
#we can also use another python command
#read_in_dataset??

### ---- 4 Reading the data ----

In [8]:
!dir -lh unzipped_data/

Invalid switch - "".


In [10]:
#Reading the data disctionary file which includes information on all features. (To be used as reference)
dataguide = pd.read_excel('zillow_data_dictionary.xlsx')
dataguide.head()

,Feature,Description
0,'airconditioningtypeid',Type of cooling system present in the home (i...
1,'architecturalstyletypeid',"Architectural style of the home (i.e. ranch, ..."
2,'basementsqft',Finished living area below or partially below...
3,'bathroomcnt',Number of bathrooms in home including fractio...
4,'bedroomcnt',Number of bedrooms in home


#### Properties

This dataset has information about the properties we'll be trying to make predictions on.

In [13]:
def read_in_dataset(dset, verbose=False):
    
    df = pd.read_csv('{0}.csv'.format(dset))
    #df = pd.read_csv('properties_2016.csv'.format(dset))
    
    if verbose:
        print('\n{0:*^80}'.format(' Reading in the {0} dataset '.format(dset)))
        print("\nit has {0} rows and {1} columns".format(*df.shape))
        print('\n{0:*^80}\n'.format(' It has the following columns '))
        print(df.columns)
        print('\n{0:*^80}\n'.format(' The first 5 rows look like this '))
        print(df.head())
        
    return df

In [14]:
properties = read_in_dataset("properties_2016", verbose=True)
properties.head()

C:\Users\TOSHIBA\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):



******************** Reading in the properties_2016 dataset ********************

it has 2985217 rows and 58 columns

************************* It has the following columns *************************

Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


#### Target variable and Transaction Date

In [15]:
train_raw = read_in_dataset("train_2016", verbose=True)


********************** Reading in the train_2016 dataset ***********************

it has 90275 rows and 3 columns

************************* It has the following columns *************************

Index(['parcelid', 'logerror', 'transactiondate'], dtype='object')

*********************** The first 5 rows look like this ************************

   parcelid  logerror transactiondate
0  11016594    0.0276      2016-01-01
1  14366692   -0.1684      2016-01-01
2  12098116   -0.0040      2016-01-01
3  12643413    0.0218      2016-01-02
4  14432541   -0.0050      2016-01-02


### ---- 5 Exploratory Data Analysis ----

In [46]:
#how many training records do not have a matching property
len(set(train_raw['parcelid'])-set(train_raw['parcelid']))

0

In [47]:
#how many properties do not have a matching training record

len(set(properties['parcelid']) - set(train_raw['parcelid']))

2895067

So there are a lot of properties that don't have a corresponding target. What could we use these records for? For now, lets separate them out so we don't have to carry them around in memory

#### Making Dataset for Analysis

There's a lot of data in the properties dataset that we won't be using (because those properties didn't sell and therefore we don't know what the target values are). To improve the speed of loading in the data and reduce the memory strain, lets create a file with only the properties and sales records we'll need for training.

In [48]:
# inspect the function we made for this
print(inspect.getsource(merge_dataset))

def merge_dataset(train, properties):
    
    """Merge the train and properties datasets. Both need to have a common key `parcelid`

    Keyword arguments:
    train -- the dataframe of transactions
    properties -- the dataframe of properties
    
    Returns:
    a pandas dataframe
    """

    train_data_merged = train.merge(properties, how='left', on='parcelid')
    
    return train_data_merged



In [50]:
def merge_dataset(train, properties):
    
    """Merge the train and properties datasets. Both need to have a common key `parcelid`

    Keyword arguments:
    train -- the dataframe of transactions
    properties -- the dataframe of properties
    
    Returns:
    a pandas dataframe
    """

    train_data_merged = train.merge(properties, how='left', on='parcelid')
    
    return train_data_merged

In [54]:
# merge the data and remove the raw datasets
train_data_merged = merge_dataset(train_raw, properties)
#del train_raw
#del properties
train_data_merged.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


Now this is what we'll call our modeling file because it has all the data needed to build the model. Let's put it in a directory called derived_data so we can read it in the next notebook.

In [60]:
# make a directory to hold our derived datasets
!mkdir -p derived_data

A subdirectory or file derived_data already exists.
Error occurred while processing: derived_data.


In [61]:
# write out the training data to csv so we can load it into the next notebook
train_data_merged.to_csv('derived_data/train_data_merged_2016.csv', index=False)